In [1]:
import numpy as np
from pyscf import gto, scf, fci
from tabulate import tabulate
from PNOFs import HF_2RDM, MU_2RDM, GU_2RDM, BBC1_2RDM, R_PNOF2
from utils import R_spectral_clean, DM2_DiagSum, R_twoDM_Eigvals
import scipy 

In [2]:
# Get Hydrogen in mo basis from PySCF

mol = gto.Mole()
mol.unit = 'B' 
L = 1.5
mol.atom = f"""
# He 0 0 0 
H   {-L}   {-L}   0.0
H    {L}   {-L}   0.0
H    {L}    {L}   0.0
H   {-L}    {L}   0.0
"""
# this basis has 2 functions for Helium
mol.basis = "ccpvdz"
# mol.basis = "sto-3g"
# mol.basis = "631g"

mol.spin =  0
mol.verbose= 0
mol.build()

# the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 entries
eri_ao = mol.intor('int2e')
S = mol.intor('int1e_ovlp')

## Run Hartree-Fock.
mf = scf.RHF(mol)
mf.kernel()


# Harvest HF quantities

h1 = mf.get_hcore()
M = h1.shape[0]
C_a = mf.mo_coeff
h1_a = C_a.T@h1@C_a
eri_h2_aa = eri_ao.copy()

print(mol.nelec)

for i in range(4):
  eri_h2_aa = np.tensordot(eri_h2_aa, C_a, axes=1).transpose(3, 0, 1, 2)


h1_mo = (h1_a )
h2_mo = (eri_h2_aa )

(2, 2)


In [3]:
dm1_ao = mf.make_rdm1()
dm2_ao = mf.make_rdm2()

# # dm1_mo=C[:,0:N]@dm1_ao@C[:,0:N].T
dm1_mo=(C_a.T@(S.T@dm1_ao@S))@C_a

dm2_mo = dm2_ao.copy()
for i in range(4):
  dm2_mo = np.tensordot(dm2_mo, S@C_a, axes=1).transpose(3, 0, 1, 2)


# P = C@C.T
# print(P)

occ, C = R_spectral_clean(dm1_mo, h1.shape[0])
print(occ, np.sum(occ))
print(dm1_mo.shape, dm1_ao.shape, C_a.shape)

[2.00000000e+00 2.00000000e+00 2.30454246e-15 1.22390827e-15
 3.12126884e-16 2.39928888e-16 6.90117144e-17 3.87005074e-17
 1.60108885e-18 1.77156083e-19 6.02325242e-26 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00] 3.999999999999999
(20, 20) (20, 20) (20, 20)


In [4]:
dm2_HF_mo = HF_2RDM(occ, mol.nao)
dm2_MU_mo = MU_2RDM(occ,  mol.nao)
dm2_GU_mo = GU_2RDM(occ,  mol.nao)
dm2_BBC1_mo = BBC1_2RDM(occ, mol.nao,mol.nelec[0])
dm2_PNOF2_mo = R_PNOF2(occ, mol.nao,mol.nelec[0])


In [5]:
print(np.sum(R_twoDM_Eigvals(np.transpose(dm2_HF_mo, (0, 2, 1, 3)))), scipy.special.binom(np.sum(mol.nelec),2))
print(np.sum(R_twoDM_Eigvals(np.transpose(dm2_MU_mo, (0, 2, 1, 3)))),scipy.special.binom(np.sum(mol.nelec),2))
print(np.sum(R_twoDM_Eigvals(np.transpose(dm2_BBC1_mo, (0, 2, 1, 3)))), scipy.special.binom(np.sum(mol.nelec),2))
print(np.sum(R_twoDM_Eigvals(np.transpose(dm2_GU_mo, (0, 2, 1, 3)))), scipy.special.binom(np.sum(mol.nelec),2))
print(np.sum(R_twoDM_Eigvals(np.transpose(dm2_PNOF2_mo, (0, 2, 1, 3)))), scipy.special.binom(np.sum(mol.nelec),2))
print(np.sum(R_twoDM_Eigvals(np.transpose(dm2_mo, (0, 2, 1, 3)))), scipy.special.binom(np.sum(mol.nelec),2))

12.000000000000005 6.0
11.999999999999995 6.0
11.999999999999995 6.0
12.000000000000002 6.0
5.999999999999997 6.0
11.999999999999972 6.0


In [11]:
# Run FCI  
cisolver = fci.direct_spin0.FCI()
cisolver.max_cycle = 100
cisolver.conv_tol = 1e-8
e, ci_mo = cisolver.kernel(h1_mo, h2_mo, h1.shape[0], mol.nelec, ecore=mf.energy_nuc())
dm1_fci, dm2_fci  = cisolver.make_rdm12(ci_mo, h1.shape[0], mol.nelec)
occ, C = R_spectral_clean(dm1_fci, h1.shape[0])
print(occ, np.sum(occ))


[1.84126337e+00 9.98674093e-01 9.98673622e-01 1.51259634e-01
 2.10753932e-03 2.10750726e-03 1.69551543e-03 1.07665095e-03
 5.86470302e-04 5.00311273e-04 5.00272422e-04 3.70006362e-04
 2.86443459e-04 2.86424506e-04 1.69899266e-04 1.24251235e-04
 1.09522699e-04 1.09518315e-04 7.54439787e-05 2.34994030e-05] 4.000000000000002


In [12]:
print(e)

-2.0714737862826027


In [13]:
dm2_HF_mo = HF_2RDM(occ, M)
dm2_MU_mo = MU_2RDM(occ, M)
dm2_GU_mo = GU_2RDM(occ, M)
dm2_BBC1_mo = BBC1_2RDM(occ,M,mol.nelec[0])
dm2_PNOF2_mo = R_PNOF2(occ, mol.nao,mol.nelec[0])


In [14]:
print(np.sum(R_twoDM_Eigvals(np.transpose(dm2_HF_mo, (0, 2, 1, 3)))), scipy.special.binom(np.sum(mol.nelec),2))
print(np.sum(R_twoDM_Eigvals(np.transpose(dm2_MU_mo, (0, 2, 1, 3)))), scipy.special.binom(np.sum(mol.nelec),2))
print(np.sum(R_twoDM_Eigvals(np.transpose(dm2_BBC1_mo, (0, 2, 1, 3)))), scipy.special.binom(np.sum(mol.nelec),2))
print(np.sum(R_twoDM_Eigvals(np.transpose(dm2_PNOF2_mo, (0, 2, 1, 3)))), scipy.special.binom(np.sum(mol.nelec),2))

print(np.sum(R_twoDM_Eigvals(np.transpose(dm2_GU_mo, (0, 2, 1, 3)))), scipy.special.binom(np.sum(mol.nelec),2))
print(np.sum(R_twoDM_Eigvals(np.transpose(dm2_fci, (0, 2, 1, 3)))), scipy.special.binom(np.sum(mol.nelec),2))

13.296078311525738 6.0
12.000000000000009 6.0
12.000000000000009 6.0
6.680593329086053 6.0
13.296078311525733 6.0
12.000000000000002 6.0
